## Filozofie projektu: Nenavist stimuluje kreativitu.

<font size="3">Tento projekt se zabyva lidmi, kteri komentuji filmu na strance CSFD.cz. CSFD je ceska obdoba stranky IMDB. 
Kazdemu registrovanemu uzivateli umoznuje ohodnotit nebo okomentovat film. Leden 2020 - 477 000 uzivatelu, 4 092 000 komentaru. CSFD umoznuje ohodnotit film jedna az peti hvezdickami. Pokud tohle nestaci, je mozne pouzit hodnoceni "odpad!", ktere byly inspiraci k vytvoreni tohoto projektu. 

Pri prohlizeni filmu s dobrymi hodnocenimi me napadlo podivat se na lidi, kteri tyto filmy nesnaseji ("odpad"). Ukazalo se, ze potreba ukazat vsem, jak moc se ve svem minieni o danem filmu pletou, vyvolava v techto lidech znacnou kreativitu pri zanechani negativniho komentare. U nekterych se clovek muze dobre zasmat ryzi zlosti autora:


_"Tvoji nenávist ke značce vody ti uvěřím, až **vymrdáš** hrdlo láhve, s tím svým tenkým **kokotem** to bude Mattoni malina. Jistě to potají chlastáš s **buzeranty**, se kterými se slézáš v barech, ty **píčo** s maskou. S tím už nepostrašíš ani dítě z mateřské školy, **pičo** ukřivděná, ubrečená, tupá. Táhni do **piče** s tou tvojí mentálně ubohou hudbou! 0%"_ - kingik

Cilem je nalezt nastvane ... zejmena u dobrych filmu.

Komentare maji sve vlastni skore, zalozene na nenavisti textu komentare. Skore se pricita za sproste slova, pouziti CAPS LOCKU a opakovani stejneho znaku ("fuuuuuuuj!!!!"). 

Krome komentaru s hodnocenim "odpad!" pracuje projekt take s hodnocenim jedne hvezdicky. Casto se v takych komentarich vyskytuje fraze: _"Jedna hvezdicka za (insert herce nebo scenu). Jinak Odpad!"_ </font>

In [23]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

import urllib
import numpy as np
import pandas as pd
from collections import Counter
from tqdm.notebook import tqdm
from time import time
from functools import wraps
import re
import pickle
import line_profiler
# %load_ext line_profiler

from selenium.common.exceptions import NoSuchElementException

from IPython.display import Markdown, display

In [24]:
def reload_driver():
    global driver
    options = Options()
    options.headless = True
    options.set_preference("javascript.enabled", False)
    driver = webdriver.Firefox(options=options)
    driver.set_page_load_timeout(100)

# reload_driver()
# options = Options()
# # options.headless = True
# options.set_preference("javascript.enabled", False)
# driver = webdriver.Firefox(options=options)
# driver.set_page_load_timeout(100)

# driver = webdriver.Firefox()

In [25]:
reload_driver()

In [94]:
# %%writefile simulation.py
# co kdyz csfd odmitne request
# hodnoceni s komentarem je zahrnoto v hodnocenich bez komentaru
# nechapu .. hodnoceni, videli ... stejny film, hvezdicku ... za -- regex


from selenium import webdriver
from selenium.webdriver.firefox.options import Options

import urllib
import numpy as np
import pandas as pd
from collections import Counter
from tqdm.notebook import tqdm
from time import time
from functools import wraps
import re
import pickle
import line_profiler

from selenium.common.exceptions import NoSuchElementException

RATING_SYSTEM = {"film c0": "white",
                 "film c1": "red",
                 "film c2": "blue",
                 "film c3": "black"}

EVIL_WORDS = {"sračk", 'stupidní', 'kretén', "kokot", "zkurvenec", 'hovno', 'hovna', 'čurák', 'hajzl', 'kurv', 'po*rať',
              'prdel', 'propagand', 'idiot', 'buzny', 'buzna', 'hnus', 'ho*no', 'hnus', 'mrtka', 'píčo',
              'blití', 'fuj', 'sra*ka', 'posrat', 'demenci', 'pí*ovina', 'píčovina', 'kravin', 'kripl',
              'vymrdaných', 'shit', 'stád', 'pí..vina', 'dement', 'zasvin', 'prase', 'úch*l', 'poser',
              'blb', 'magor', 'magoř', 'sračku', 'mrd*ka', 'držka', 'debil', 'retard', 'neger', "hovnům", 'zmrd','hovně',
             'pi*ovina', 'kund', 'piče', 'piči', 'mrd', 'pičo', 'píčo', 'píče', 'negr', 'pičus', 'sran', 'buzerant',
             'ošukán', 'posral', 'čuráci', 'serete', 'šukat', 'pošuk', 'pošuc'}

MISTAKE_WORDS = {'knihovna', 'knihovny', 'filozofující', 'sranda', 'srandu', 'srandy'}

class Comment:
    def __init__(self, rating, text, approximate_movie_rating, movie_name, author=None, movie_rating=None):
        self.rating = rating
        self.text = text
        self.approximate_movie_rating = approximate_movie_rating
        self.movie_rating = movie_rating
        self.movie_name = movie_name
        self.author = author
        self.score = self.evil_score()
        
    def add_author(self, author):
        self.author = author
        
    def evil_score(self):
        score = 0
        for word in self.text.split():
            for EVIL in EVIL_WORDS:
                if EVIL in word.lower() and word.lower() not in MISTAKE_WORDS:
                    score += 2
                    
            repeated_char, number_of_repeats = maxRepeating(word)
            if number_of_repeats > 2: # 0,33 zvoleno jen tak for fun
                number_of_repeats = 5 if number_of_repeats > 5 else number_of_repeats
                score += 0.33 * number_of_repeats
                if repeated_char == "!":
                    score += 0.33 * number_of_repeats
                
            if caps_lock(word):
                score += 0.5
        
        return score

    def to_dict(self): # pro pandas, kdyby nahodou
        return {
            'author': self.author,
            'rating': self.rating,
            'text': self.text,
            'hate_score': self.score,
            'movie_name': self.movie_name,
            'movie_rating_apr': self.approximate_movie_rating
        }


def timing(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        start = time()
        result = f(*args, **kwargs)
        end = time()
        print('Elapsed time: {}'.format(end-start))
        return result
    return wrapper


def printmd(string):
    display(Markdown(string))

        
def get_best_movie_score(driver):
    """
    Vetsina filmu nema skore nejlepsich filmu
    """
    try:
        return driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[2]/div[1]/p/a[1]')
    except:
        return None

    
def get_favourite_movie_score(driver):
    """
    Vetsina filmu nema skore oblibenosti
    """
    try:
        return driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[2]/div[1]/p/a[2]')  
    except:
        return None    

    
def get_full_czech_name(driver):
    return driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/h1')


def get_comment_text(comment):
    return comment.find_element_by_class_name('post').text


def get_comment_author(comment):
    return comment.find_element_by_class_name('author')


def get_czech_name_in_comment(comment):
    return comment.find_element_by_class_name('author').text


def get_comment_rating(comment):
    """
    return None pro komentar bez hodnoceni
           0 pro komentar typu odpad!
           pocet hvezdicek jinak
    """
    try:
        author_rating = comment.find_element_by_class_name('rating').get_attribute('alt')
    except:
        return None
    
    if not author_rating:
        return 0
    
    return len(author_rating)
   
    
def remove_last_line_from_string(s):
    return s[:s.rfind('\n')]


def count_in_string(string, char):
    return strin.count(char)


def movie_rating_to_stars(rating):
    if rating == 0:
        return "odpad!"
    else:
        return rating * "*"
    

def get_approx_movie_rating_from_comment(comment):
    """ Z komentare v profilu nalezne priblizne hodnoceni filmu podle barvy ikony
    
    Parameters:
        comment(webdriver.element?): comment inside users profile
    
    Returns:
        string:color of movie icon
    
    """
    movie_rating = comment.find_element_by_class_name("film")
    return RATING_SYSTEM[movie_rating.get_attribute("class")]


def profile_comments(driver):
    """ Nacte vsechny
    """
    current = driver.current_url
    try:
        driver.get(current + 'komentare/podle-rating/')
        
    except TimeoutException as e:
        print(e)
        driver.execute_script("window.stop();")
    
    try:
        pages = driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[2]/div[1]/div[3]/div[3]').text
    
    except NoSuchElementException: # pokud ma pouze jednu stranku komentaru
        comment_objects = profile_comments_on_page(driver, current + 'komentare/podle-rating/')
        comment_objects = [_ for _ in comment_objects if _ is not None]
        list(map(lambda c:c.add_author(current), comment_objects))
        return comment_objects
    
    pages = re.findall(r'\d+', pages)[-1] # hack, posledni cislo v seznamu, potrebuje poradne otestovat(vypada OK)
    try:
        pages = int(pages)
    except:
        raise ValueError("Neco divneho s typem")
        
    next_run = True
    comment_objects = []
    while next_run:
        local_comments = profile_comments_on_page(driver, current + 'komentare/podle-rating/strana-' + str(pages))
        if len(local_comments) == 0: # pokud nejsou zadne komentare s 0 nebo 1 hvezdou
            next_run = False
            break
            
        local_comments = [c for c in local_comments if c is not None]
        list(map(lambda c:c.add_author(current), local_comments))
        
        comment_objects = comment_objects + local_comments
        pages -= 1 
        
    return [c for c in comment_objects if c is not None] # nevim proc


def profile_comments_on_page(driver, url):
    """ Nacte komentare na strance s cislovanim
    """
    driver.get(url)
    comments = driver.find_elements_by_class_name("post")
    
    out = []
    for comment in comments:
        comment_rating = get_comment_rating(comment)
        
        if comment_rating is None:
            out.append(None) # HACK
            continue
        if comment_rating >1:
            continue

        movie_approx = get_approx_movie_rating_from_comment(comment)
        if movie_approx != 'red':
            continue
            
        comment_text = comment.text.split('\n')[1:][0] # hack, prvni radek neni text komentare
        comment_text = comment_text.replace("*", "#") # nahrazeni hvezdicek, ktere se v komentari vyskytuji
        
        
        movie_name = get_czech_name_in_comment(comment)

        comment_object = Comment(comment_rating, comment_text, movie_approx, movie_name)
        out.append(comment_object)
    
    return out


def maxRepeating(string): 
    """ Najde znak, ktery se nejvicekrat vyskytuje za sebou ve stringu 
    Return: tuple (znak, pocet opakovani)
    """
  
    n = len(string) 
    count = 0
    res = string[0] 
    cur_count = 1
  
    for i in range(n):
        if (i < n - 1 and string[i] == string[i + 1]): 
            cur_count += 1
  
        else: 
            if cur_count > count and string[i] != ".": 
                count = cur_count 
                res = string[i] 
            cur_count = 1
    return res, count


def caps_lock(string):
    if len(string) > 2 and string.isupper():
        return True


def comment_split(comment_text): # pouzite jen k vytvoreni counteru slov
    words = []
    for word in comment_text.split():
        if word[-1] == "," or word[-1] == ".":
            word = word[0:-1]
            
        word = word.lower()
        words.append(word)
        
    return words
    
    
def replace_in_comment(comment_text, replace): # vic testovani!!
    words = comment_text.split()
    for i,word in enumerate(words):
        for EVIL in EVIL_WORDS:
            if EVIL in word.lower():
                words[i] = replace + word + replace
    return " ".join(words)

In [112]:
class Profile:
    def __init__(self, link, num_of_comments, num_of_ratings, comments, score):
        self.link = link
        self.id = self.find_id()
        self.num_of_comments = len(self.comments)
        self.num_of_ratings = num_of_ratings
        self.comments = comments
        self.score = score
        
    def find_id(self):
        id_re = re.match(".*/uzivatel/(\d+)", self.link)
        try:
            aids = int(id_re.group(1))
            return aids
        except:
            raise ValueError("TOHLE BY SE NEMELO STAT U NORMALNIHO URL")
            
    def __repr__(self):
        return "{}, Pocet komentaru: {}, skore: {}".format(self.link, self.num_of_comments, self.score)
    
    def to_dict(self): # pro pandas
        return {
            'id': self.id,
            'num_of_comments': self.num_of_comments,
            'score' : self.score
        }
            

# profile_comments('https://www.csfd.cz/uzivatel/363-woody', driver)

def profile_ratings(driver):
    
    current = driver.current_url
    driver.get(current + '/hodnoceni/')
    
    ratings = driver.find_elements_by_class_name('rating')
    
    all_ratings = []
    for rating in ratings:
        rating = rating.get_attribute('alt')
        if not rating:
            all_ratings.append(0)
        else:
            all_ratings.append(len(rating))
#         print(rating.get_attribute('alt'))
    
    next_run = True
    i = 2
    print("Prochazim stranky s hodnocenim")
    while next_run:
        driver.get(current + 'hodnoceni/strana-' + str(i) + '/')
        ratings = driver.find_elements_by_class_name('rating')
        i += 1
        
        if len(ratings) == 0: # prazdna stranka
            next_run = False
            continue
        
        
        for rating in ratings:
            rating = rating.get_attribute('alt')
            if not rating:
                all_ratings.append(0)
            else:
                all_ratings.append(len(rating))

    return all_ratings

# @timing
def profile_summary(url, print_command=False): # co kdyz nekdo zada platnou adresu ale nebude tam profil ... checknout url
    
    driver.get(url)
    
    try:
        error = driver.find_element_by_id('pg-web-error') # nejake divne 
        if error:
#             print(error.text)
            raise ValueError("Neplatna url adresa")
    except NoSuchElementException:
        pass
    
    
    comments = profile_comments(driver)
    
    if print_command:
        print("Profil:", url)
        print("Nenavistne komentare u filmu/serialu s vetsim hodnocenim nez 70%")
        print()
    
    evil_comments = Counter()
    sum_of_comment_score = 0
    for comment in comments:
        
#         sum_of_comment_score += comment.score # # bud tady pro vsechny filmy
        if comment.approximate_movie_rating == "red": # pokud hodnoti cerveny film odpadem nebo 1 hvezdou - melo by jit bez if
            
            comment_text = comment.text
            
            printed = False
            comment_text = replace_in_comment(comment_text, '**')
            comment.text = comment_text
            if "**" in comment_text:
                printed=True
            
            sum_of_comment_score += comment.score # # nebo tady pro spatne hodnocene
            if printed and print_command:
                
                print(comment.movie_name, movie_rating_to_stars(comment.rating), comment.score)
                printmd(comment_text)
                print()
                
            comment_words = comment_split(comment.text)
            for word in comment_words:
                evil_comments[word] += 1
            
    
    if print_command:
        print("Skore profilu:", round(sum_of_comment_score,1))
        print("Pomer unikatnich a vsech slov", len(evil_comments)/sum(evil_comments.values()))
        
    profile_object = Profile(url, 0, comments, sum_of_comment_score)
    return profile_object
#     print(evil_comments)
#     driver.get(url)
#     ratings = profile_ratings(driver)
    
#     counter = Counter(ratings)
#     print('Prumerne hodnoceni {} hvezdicky'.format(round(np.mean(ratings),1)))
#     for i in range(5,-1,-1):
#         if i != 0:
#             print('{:>6} {:>5}'.format(i*'*', counter[i]))
#         else:
#             print('{:>6} {:>5}'.format('odpad!', counter[i]))

In [96]:
class Movie:
    
    def __init__(self, czech_name, rating, comments, best_movie = None, fav_movie = None):
        self.czech_name = czech_name
        self.rating = rating
        self.comments = comments
        self.best_movie = best_movie
        self.fav_movie = fav_movie
        self.hate_score = self.hate()
        
    def hate(self):
        if len(self.comments) == 0:
            return 0
        else:
            total = 0
            for comment in self.comments:
                total += comment.score
                
            return total/len(self.comments)
        
    def __repr__(self):
        return '{} Hodnoceni: {} Pocet nenavistnych komentaru: {}, Nenavist skore: {}'.format(self.czech_name, 
                                                                            self.rating,
                                                                            len(self.comments),
                                                                           round(self.hate_score, 2))

    
def get_transformed_comment(comment):
    comment = get_comment_text(comment)
    comment = remove_last_line_from_string(comment) # posledni radek je datum komentare
    comment = comment.replace("*", "#")
    comment = replace_in_comment(comment, '**')
    return comment
    

@timing
def movie_summary(url, print_command=False):
    """
    finds 
    """
    
    if '/prehled/' in url:
        url = url.replace('/prehled/', '/')
    
    driver.get(url+'/komentare/podle-rating/?all=1')
    
    full_czech_name = get_full_czech_name(driver).text
    movie_rating = driver.find_element_by_class_name('average')
    
    best_movies = get_best_movie_score(driver)
    favourite_movies = get_favourite_movie_score(driver)
    
    if favourite_movies:
        favourite_movies = favourite_movies.text
    if best_movies:
        best_movies = best_movies.text
    
    comments = driver.find_elements_by_xpath('//*[starts-with(@id, "comment")]')
    
    comment_objects = []
    for comment in tqdm(reversed(comments)):
        rating = get_comment_rating(comment)
        
        if rating is None:
            continue
             
        # zrychli vypocet, ale prijde o schopnost vypsat counter hodnoceni
        if rating > 1:
            break
        
        trash_comment = get_transformed_comment(comment)
        author = get_comment_author(comment)
        profile_link = comment.find_element_by_css_selector(".author [href]").get_attribute('href')

        comment_object = Comment(rating, trash_comment, 'red', full_czech_name, author = profile_link) # FIX
        comment_objects.append(comment_object)
        
    comment_objects.sort(key=lambda x: x.score, reverse=True)
    
    movie_object = Movie(full_czech_name, movie_rating.text, comment_objects, best_movies, favourite_movies)
    
    if print_command:
        print(full_czech_name)
        print("Hodnoceni filmu: {}".format(movie_rating.text)) # co kdyz zatim nema rating
        print("Hodnoceni pouze z komentaru: {}%".format(int(round(np.mean(avg)*20,0)))) # hodnoceni lidi s komentari muze byt rozdilne od vsech hodnoceni
        print()
        if favourite_movies:
            print(favourite_movies)
        if best_movies:
            print(best_movies)
            
        print('Pocet komentaru: {}'.format(len(comments)))
        print('Pocet komentaru odpad nebo jedna *: {}'.format(len(comment_objects)))
        print()
                
            
    return movie_object
    

## Vytahovani informaci ze zebricku

In [67]:
class leaderboard_item:
    
    def __init__(self, order, name, link, rating, number_of_ratings):
        self.order = order
        self.name = name
        self.link = link
        self.rating = rating
        self.number_of_ratings = number_of_ratings
        
    def __repr__(self):
        return  '{} {}, {}, {}'.format(self.order, self.name, self.rating, self.number_of_ratings)

def csfd_leaderboards(type_of_board): 
    
    url = ('https://www.csfd.cz/zebricky/' + type_of_board + '?show=complete')
    driver.get(url)
    items = driver.find_elements_by_xpath('/html/body/div[2]/div[3]/div[2]/table/tbody/tr')
    
    leaderboard_items = []
    for item in tqdm(items):
        
        try:
            order = item.find_element_by_class_name('order').text
            name = item.find_element_by_class_name('film').text
            link = item.find_element_by_css_selector(".film [href]").get_attribute('href')
            rating = item.find_element_by_class_name('average').text
            number_of_ratings = item.find_element_by_class_name('count').text
            
            board_item = leaderboard_item(order, name, link, rating, number_of_ratings)
            leaderboard_items.append(board_item)
            
        except NoSuchElementException: # pravdepodobne prazdny <tr> element
            continue
            
    return leaderboard_items


# leaders = csfd_leaderboards('nejlepsi-filmy/')

## Ukladani a nacitani ze souboru

In [28]:
def get_profiles_from_urls(profile, ids):
    """ Z url adresy vrati profil objekt
    """
     # odstraneni duplikatu (udelat lip, tohle nacita cely profil a az potom checkuje)
    print(profile)
    profile = profile_summary(profile)

    if profile.id not in ids:
        return profile
    else:
        return None
        

def store_id(ids):
    """ Ulozi pole idecek do souboru"""
    with open('ids.txt', 'a') as file:
        for id in ids:
            file.write("{} ".format(id))
            
def load_ids():
    """ Nacte idcka ze souboru"""
    out = []
    with open('ids.txt', 'r') as f:
        content = f.readlines()
        if content:
            for num in content[0].split():
                out.append(int(num))

    return out
# profile_objects = get_profiles_from_urls(worst_profiles)

def store_profiles(profile):
    """ ulozi profilove objekty do souboru"""
    if profile is None:
        raise ValueError("None misto profilu")
        
    with open('profiles.pkl', 'ab') as output:
        add_id = []
        
        ids = load_ids() # nacteni souboru s IDcky
            
        if profile.id not in ids:
            pickle.dump(profile, output, pickle.HIGHEST_PROTOCOL)
            add_id.append(profile.id)
            print('ADDING: ',profile.id)
                
        store_id(add_id)
        

def load_profiles():
    """ Nacte vsechny profily ze souboru zpatky jako objekty"""
    objs = []
    with open('profiles.pkl', 'rb') as f:
        
        while 1:
            try:
                o = pickle.load(f)
            except EOFError:
                break
            objs.append(o)
    return objs
        
# load_profiles()
def add_interesting_user(url, note=None):
    with open('interesting.txt', 'a') as file:
        file.write(url)
        if note is not None:
            file.write(' ## '+note)
        file.write('\n')

## Vysledky

In [ ]:
movies = []
for l in leaders[40:60]:
    print(l)
    movies.append(movie_summary(l.link))
    
    

In [ ]:
worst_profiles = []
aids = load_ids()

for movie in movies:
    comments = movie.comments
    print(movie.czech_name)
    for comment in comments:
        if comment.score > 1:
            id_re = re.match(".*/uzivatel/(\d+)", comment.author)
            try:
                id_re = int(id_re.group(1))
            except:
                raise ValueError("TOHLE BY SE NEMELO STAT U NORMALNIHO URL")
            
            if id_re not in aids:
                print(comment.author,comment.score)
                worst_profiles.append(comment.author)
                printmd(comment.text)
            
    print()

In [80]:
def print_stored_profiles():
    movie_counter = Counter()
    stored_profiles = load_profiles()
    for profile in stored_profiles:
        if profile.score > 1:
            print(profile.link, profile.score)
            comments = profile.comments
            for comment in comments:
                movie_counter[comment.movie_name] += 1
                
    print(movie_counter.most_common(30))
        
print_stored_profiles()

https://www.csfd.cz/uzivatel/653636-panfilmu/ 32.12
https://www.csfd.cz/uzivatel/138677-c0r0ner/ 127.99
https://www.csfd.cz/uzivatel/352146-foogas/ 11.49
https://www.csfd.cz/uzivatel/172466-frajer42/ 1686.76
https://www.csfd.cz/uzivatel/516743-buchannon/ 125.99000000000001
https://www.csfd.cz/uzivatel/181204-mikulisek/ 4
https://www.csfd.cz/uzivatel/34405-merggie/ 1.98
https://www.csfd.cz/uzivatel/242454-maq/ 547.320000000001
https://www.csfd.cz/uzivatel/62237-carthorias/ 30.470000000000002
https://www.csfd.cz/uzivatel/360367-desmi/ 84.41000000000001
https://www.csfd.cz/uzivatel/214956-karel321/ 62
https://www.csfd.cz/uzivatel/466252-ozzy2/ 252.34000000000003
https://www.csfd.cz/uzivatel/76909-burygol/ 10
https://www.csfd.cz/uzivatel/144208-biopler/ 24.0
https://www.csfd.cz/uzivatel/666970-megahead/ 2
https://www.csfd.cz/uzivatel/354044-damnsinead/ 8.65
https://www.csfd.cz/uzivatel/437840-dominique55/ 22
https://www.csfd.cz/uzivatel/38661-jarvis/ 11.47
https://www.csfd.cz/uzivatel/2102

In [61]:
ids = load_ids()
# print(worst_profiles[23])
# tmp =get_profiles_from_urls(worst_profiles[23], ids)
# try:
#     store_profiles(tmp)
# except ValueError as e:
#     print(e)
for worst in tqdm(worst_profiles):
    tmp = get_profiles_from_urls(worst, ids) # osetrit duplikaty ve worst, bude vracet None pokud id profilu bude v ids
    try:
        store_profiles(tmp)
    except ValueError as e:
        print(e)

https://www.csfd.cz/uzivatel/304734-cubalibrecz/
ADDING:  304734
https://www.csfd.cz/uzivatel/290063-kiv/
ADDING:  290063
https://www.csfd.cz/uzivatel/749174-redeaglesk/
ADDING:  749174
https://www.csfd.cz/uzivatel/93823-dezert/
ADDING:  93823
https://www.csfd.cz/uzivatel/500095-robfi/
ADDING:  500095
https://www.csfd.cz/uzivatel/576259-mamutol/
ADDING:  576259
https://www.csfd.cz/uzivatel/182277-tryskomys/
ADDING:  182277
https://www.csfd.cz/uzivatel/206100-nicneznam/
ADDING:  206100
https://www.csfd.cz/uzivatel/45228-vironey/
ADDING:  45228
https://www.csfd.cz/uzivatel/360129-jarmila68/
ADDING:  360129
https://www.csfd.cz/uzivatel/455122-pantokrator/
ADDING:  455122
https://www.csfd.cz/uzivatel/582917-sssiksseilor/
ADDING:  582917
https://www.csfd.cz/uzivatel/136195-phoenixer/
ADDING:  136195
https://www.csfd.cz/uzivatel/266561-misbos/
ADDING:  266561
https://www.csfd.cz/uzivatel/77252-mraldridge/
ADDING:  77252
https://www.csfd.cz/uzivatel/497312-tenpravy/
ADDING:  497312
https://ww

In [ ]:
'knihovny', 'průser', 'Serete', 'filozofující' - fuj, citace z filmu

## Analyzy

In [107]:
stored_profiles = load_profiles()

In [110]:
def comments_to_dataframe():
    all_comments = []
    for profile in stored_profiles:
        comments = profile.comments
        all_comments = all_comments + comments
    
    frame = pd.DataFrame.from_records([comment.to_dict() for comment in all_comments if comment.rating <2 and comment.approximate_movie_rating == "red"])
#     frame = frame.sort_values(by=['hate_score'], ascending=False)
    return frame
    
    
def profiles_to_dataframe():
    frame = pd.DataFrame.from_records([profile.to_dict() for profile in stored_profiles])
    return frame

    
# profiles_to_dataframe()

In [111]:
profiles_frame = profiles_to_dataframe().sort_values(by=['score'], ascending=False)
profiles_frame

,id,num_of_comments,score
3,172466,0,1686.76
184,163977,0,956.52
89,141370,0,922.54
29,142650,0,585.68
72,26324,0,550.06
...,...,...,...
124,366041,0,0.00
131,221906,0,0.00
18,15559,0,0.00
314,63075,0,0.00


## Testovani konkretnich filmu a profilu

In [54]:
movie_summary('https://www.csfd.cz/film/252669-temny-rytir-povstal/prehled/', print_command=True)

Temný rytíř povstal
Hodnoceni filmu: 86%
Hodnoceni pouze z komentaru: 15%

41. nejoblíbenější film
131. nejlepší film
Pocet komentaru: 3021
Pocet komentaru odpad nebo jedna *: 91

Vycet hodnoceni:
 *****     0
  ****     0
   ***     0
    **     1
     *    65
odpad!    26
Elapsed time: 12.288005352020264


Temný rytíř povstal Hodnoceni: 86% Pocet nenavistnych komentaru: 91, Nenavist skore: 2.22

In [77]:
profile_summary('https://www.csfd.cz/uzivatel/403723-jason-x/', print_command=True)

Profil: https://www.csfd.cz/uzivatel/403723-jason-x/
Nenavistne komentare u filmu/serialu s vetsim hodnocenim nez 70%

Pasažéři (2016) odpad! 2


Vysokoprocentní, ne-li koncentrovaný filmový hnůj, lovestory pro mentálně postižené, romantika pro chudé duchem. Čekal jsem nějaký průser, ale tohle předčilo i moje nejdivočejší představy. Dost možná vrchol demence moderní sci-fi scénáristiky, tedy "sci-fi"... Už jsem tu naplil na hodně hollywoodského pseudosci-fi svinstva, ale tenhle **retardovaný** potrat je zatím bez konkurence a zdaleka předčí všechna Elysia a jiný podobný komerční škvár. Uzavřu s panem rejžou Tydlydumem takovou tichou dohodu - pokud se někdy v budoucnosti objeví na filmovém sci-fi trhu ještě větší humáč než jeho Passengers, vrátím se sem a opravím moje hodnocení na aspoň jednu hvězdu za vizuál (za co taky jiného). Ale troufnu si směle tipnout, že se tak nikdy nestane.


Jaffa (2009) odpad! 2


Přiznávám, že můj odpad! je čistě tendenční ohodnocení, protože Jaffa je taktéž tendenční film. Co jím chtěla "feministka a politická aktivistka" Jedaja sdělit? Že izraelská společnost je nespravedlivá a plná předsudků vůči tamním Arabům? Božínku, a neplatí to snad desateronásobně i obráceně? A tahle patetická a notně přihlouplá mezináboženská lovestory se šablonovitými postavami má snad ony předsudky oslabit? Myslím, že většinu běžných izraelských Židů musí podobný snímek spíš znechutit, stejně jako mě třeba znechucují plky všelijakých pseudohumanistických pravdoláskařů, kteří hystericky hájí práva do Evropy přivandrovalých muslimů na úkor práv nás Evropanů. Ale přesně takovéhle **sračky** mají zjevně festivaloví diváci rádi, co naplat, kdo chce kam...


Ona (2013) odpad! 3.5


...


Gladiátor (2000) * 6


(Extended version) Ani druhá projekce mě nepřiměla ke smířlivosti, k toleranci. Spíš naopak, jsem znechucený ještě víc. Jediné co z filmu jakž takž za něco stojí, je jeho vizuální stránka, výpravu holt umějí hollywoodští profíci dobře. Stejně tak dobře ovšem umějí vyrábět **retardované** scénáře pro tupé divácké masy. Ale jistě, mají recht, většinu (naprostou většinu) diváků bude asi stěží zajímat, nakolik je děj filmu scénáristický výcuc a nakolik odpovídá skutečnosti. Takže proto tahle telenovelicky senilní slátanina může slavit takový úspěch. Kdyby byl děj zasazený do fiktivních reálií s fiktivními postavami, ocením ho jako slušné drámko o křivdě a pomstě s několika hezkými bojovými scénami, i když zmršenými divokým střihem a skákající kamerou. Takhle ho ovšem poprávu "oceňuju" jako výplod ziskuchtivých komediantských degenů, jako ostudné poplivání historických faktů, jako urážku římské historie, jako ten nejnižší druh filmové **prasečiny,** podbízivé a líbivé. Nesnáším podobné mainstreamové "velkofilmy", nesnáším jejich **stupidní** a stereotypní schémata, nesnáším jejich cajdákovitou hudbu (Zimmer neZimmer). Každé balkánské céčko s tlustým Seagalem je mi stokrát milejší. Nevermore, moje 'potřetí' už nikdy nebude.


MOST! (TV seriál) (2019) * 4.5


(O tomhle seriálu jsem se dozvěděl od chlapů z práce, protože na TV už spoustu let vůbec nečumím. Zhlédnuto na YT.) Myslím, že po třech dílech už mám celkem jasno. Zatímco při dílu prvním mohl mít divák pocit, že by se to mohlo postupně vyvrbit v docela zemitou a všestranně nekorektní bejkárnu, i když samozřejmě v mezích zákona (žijeme přece v **předposrané** Čechárně), vrbí se to postupně v další buranskou, infantilní, laskavě hřejivou, "nenápadně" moralizující šaškárnu, s tradičně hezky česky "úsměvnými" (= trapnými) karikaturami malých českých lidiček, určenou uťápnutým jednoduchým taťkům a mamkám na večer k 'televízi', aby se u ní mohli smát blahem s držkou rozšklebenou od ucha k uchu, cpát se při tom knedlíky a napájet pívem. Při sledování podobných typicky českých **debilit** mám už od mala obtížně definovatelný pocit svíravé trapnosti, studu za to, že i já sám jsem příslušníkem tohohle národa. Divím se jen, že s něčím takovým nepřišla spíš plebejská Nova. Provizorně zaokrouhluju dolů a uvidím na konci (jestli ho vůbec uvidím). // Po proklikání čtvrtého dílu - ne, neuvidím, odmítám u toho chcípnout trapností.


Skore profilu: 201.0
Pomer unikatnich a vsech slov 0.5415479582146249


In [78]:
profil = profile_summary('https://www.csfd.cz/uzivatel/149092-schuldiner/', print_command=True)

Profil: https://www.csfd.cz/uzivatel/149092-schuldiner/
Nenavistne komentare u filmu/serialu s vetsim hodnocenim nez 70%

Braindead - Živí mrtví (1992) odpad! 2


Slátanina, **kravina...** Ani komedie, ani horor... Neni to moc vtipný, člověk se u toho ani nelekne... Je to parodie, ale dost nudná! Jen spousta krve, slizu a jinejch různejch sraček..ze kterejch mi bohužel ani nebylo špatně... Prostě velký nic! Ani nevim proč jsem to dokoukal...


Pán velryb (2002) odpad! 2


Nuda jako **prase!** Tenhle film mě prostě totálně minul.. ani se to nedalo dokoukat..


Z deníku žáka III. B aneb Edudant a Francimor (TV seriál) (1993) odpad! 5.960000000000001


...


Gangster Ka (2015) * 2


Trapné! Pokud čeští tvůrci chtějí udělat akční nebo gangsterský film, dopadne to vždycky naprosto **idiotsky!** Až je mi to vážně líto.. Strašné, jak moc se snaží to udělat kultovní. V podstatě vždycky je to ale jen kopírka nějakých už dávno viděných standardů. Originalita žádná. O to ale tolik nejde, jen kdyby si to alespoň na nic nehrálo. Tenhle film mě tedy alespoň úplně nenudil. Ale to je jako plus docela málo:-) Nic jiného tam vypíchnout nelze. A kdybych měl vyloženě kritizovat, upsal bych se k smrti.. Protože toho je fakt hromada! Hromada trapných věcí, trapných lidí a trapných událostí...


Příběh kmotra (2013) * 6


Tak tahle rádoby gangsterka, která je jakože podle skutečných událostí, bohužel s jinými jmény dotyčných osob, je jedna velká katastrofa! Hrozně mě rozesmívá ta fakt až komická doslovnost v dialozích. To asi aby to pochopil i naprostý **idiot.** Tohle ale není bůhvíjak složitý film a děj, tady se fakt nikdo neztratí.. Nejde jen o doslovnost, jde o úplně nový rozměr, vytvořený tak, aby se do každého rozhovoru vešlo všechno důležité a bylo to naprosto nepochybně jasné a beznadějně nematoucí. Nemám rád ani úplný opak, tedy když je něco okatě, jakože efektně, matoucí. Ale tohle? Tahle režie? Tak to je snad vtip! Bohužel to s reálnými lidskými rozhovory a emocemi má společného fakt málo. Působí to, jak jsem již bystře podotkl, dosti komicky, až si člověk ťuká na čelo jestli to fakt myslí s tou doslovností a zároveň zkratkovitostí vážně... Hodně špatný film. Herecky taktéž... velmi **debilní** ..takzvaně **pičou** ke zdi! Češi prostě kriminálky absolutně neumí!


Skore profilu: 338.9
Pomer unikatnich a vsech slov 0.3513833566584132


In [207]:
for comment in profil.comments:
    print(profil.link)
    if comment.score > 1:
        print(comment.movie_name,comment.score)
        printmd(comment.text)
        print()

https://www.csfd.cz/uzivatel/172466-frajer42/
Rec 2 (2009) 4


Po fatálně **blbém** prvním dílu jsem si řekl, že druhý díl nemůže být horší. Spletl jsem se. Stupidita prvního dílu nabrala ještě větších rozměrů. Horor? V žádném případě. Tento film má blíže ke komedii, i kdyby jen proto, že v situacích, kde jde o holý život, tak je první a jediné co postavy zajímá kamera. Primárně vše zdokumentovat. **Sekundárně** se zachránit. Při té příležitosti, taky naní na škodu si za pomocí směšného křížku pokecat s ďáblem. (Režisér, dá - li se tak osoba zodpovědná za tento nesmysl nazvat, zřejmě hledal inspiraci ve vymítači ďábla.) Pevně doufám, že se nedočkáme trilogie. Těžko říct, kde by se tahle sračička ze Španělska mohla až zastavit.


https://www.csfd.cz/uzivatel/172466-frajer42/
Mrtvá nevěsta Tima Burtona (2005) 2.5


Trapné, jako strčit šroubovák do zásuvky. Jeden z mála filmů, kde plně docením krátkou stopáž.(Déle bych to nevydržel.) Zajímavý barevný přechod mezi životem a smrtí (viz. POMO). Méně zajímavý děj a **stupidní** pointa aneb a žili šťastně až do smrti ( v tomto případě zřejmě i po ní). Tim Burton by mohl sloužit jako reklama na nepřímou úměrnost. Čím více filmů od něj člověk vidí tím méně ho uznává.


https://www.csfd.cz/uzivatel/172466-frajer42/
Grindhouse: Planeta Teror (2007) 2.5


...

Otravná, více než 2hodinová konverzační love story. Přesně ten typ filmu, které bytostně nenávidím. Totální nuda k uzoufání. Pokud jste si stejně jako já až dosud mysleli, že Kevin Costner neumí hrát špatně, tak jste se hrubě zmýlili. Tato role mu vůbec nesvědčila. Ačkoli se snímek každým coulem snaží tlačit na emoce, tak mě velkým obloukem minul. Chemie mezi Costnerem a Kelly Preston totálně nefunguje. Dialogy jsou směšně **debilní.** Jediné, co stojí za pozornost je samotný sport a dění na hřišti. To je ovšem děsně upozaděno za **debilní** a nudnou romantickou linkou v popředí. Dva lidé, kteří se k sobě naprosto nehodí se pokoušejí budovat vztah. Největším kamenem úrazu je to, že celé budouvání vztahu je nesmírně stereotypní a především totálně klišoidní. Stejný průběh vztahu už člověk viděl milionkrát. Tohle se Raimimu hrubě nevydařilo a považuji to za jeho dosud nejubožejší film. Naprosto nezáživný pohled do života sportovní hvězdy, který je stejně prázdný, jako tento film.


https://www.csfd.cz/uzivatel/172466-frajer42/
Život pod vodou (2004) 4


Po technické stránce nemám co namítnout. Naopak se mi nesmírně líbila kamera i hudba. Co se ovšem týče obsahu, tak jsem byl nefalšovaně **zhnusen,** protože něco tak extrémně odporného a především nudného se běžně nevidí. Film je samozřejmě klasická Andersonovka s množstvím divných postav, ale zatím jsem od něj viděl snímky, které měly alespoň malý potenciál vtáhnout diváka do děje. V případě tohoto filmu jsem měl pocit, že se snímek naopak vehementně snaží diváky z děje vypudit. Děje se totiž úplné **hovno** a děje se to vy vysoce infantilním stylu. Nezasmál jsem se během stopáže ani jednou. Nic proti bizarnosti a šílenosti, ale když už se o to autor pokouší, ať to alespoň dotáhne na maximum. Zde mi to celé přišlo ušité trošku horkou jehlou a výsledek na mě vůbec nezapůsobil. Jednu hvězdu za originalitu, které bylo Andersonovi naděleno do vínku opravdu hodně.


https://www.csfd.cz/uzivatel/172466-frajer42/
https://www.csfd.cz/uzivatel/172466-frajer42/
https://www.csfd.cz/uzivatel/172466-frajer42/
Mluv s ní (2002) 2


Musím se přiznat, že tentokrát se mi do vkusu Pedro úplně netrefil. Zřejmě nikdo, kdo si vyloženě nelibuje v nekrofilii si nepřijde na své. Snímek jsem musel sledovat na tři fáze, neboť má neskutečně ospalé tempo. Zejména první polovina filmu je velmi otravná, protože divák nemá šanci chápat motivace hlavních postav, které se odkrývají velice pozvolna a po celou stopáž. Každých 10 minut někdo brečí, což je horší než v nějaké turecké telenovele. Enormně nesnáším ten **debilní** španělský sport "býčí zápasy", kde hrozní odvážlivci vstupují do boje s polomrtvým zvířetem. Považuji to za záležitost pro lidi, kteří nejsou odvážní, ale rádi by byli a srdce mi zaplesá, když je tu a tam nějaký toreador odnášen v igelitovém pytli. Totálně jsem v celém filmu postrádal na své straně jakékoli emoce. Ocenit tak mohu snad pouze dost neotřelé zpracování, vtipnou sexuální scénku a to, že si snímek nehraje na pohádku, nýbrž ukazuje život takový, jaký může být a jaký velice často bývá. V tomto směru Pedro nezklame. Lepší hvězdička.

In [75]:
add_interesting_user('https://www.csfd.cz/uzivatel/749174-redeaglesk/', 'Originalni, casto pise o Shemale')